In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os
import requests
# 🚀 **1. Đọc dữ liệu từ file CSV**
file_path1 = "C:\\Users\\PC\\Desktop\\RIOT\\top600_mid_players.csv"
file_path2 = "C:\\Users\\PC\\Desktop\\RIOT\\next1000_mid_players.csv"

df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df = pd.concat([df1, df2], ignore_index=True)

# 🚀 **2. Định nghĩa vai trò tướng**
champion_roles = ["Assassin", "Mage - Mid", "Mage - Support", "Fighter", "Marksman", "Tank"]

champion_roles1 = {
    "Assassin": ["Akali", "Zed", "Talon", "Katarina", "Qiyana", "Ekko", "Nocturne",
                 "Fizz", "Gragas", "Ahri", "Akshan", "Aurora", "Naafiri", "Pantheon", "Viego"
                 ,"Kassadin","Nidalee","Pyke","Sylas"],
    "Mage - Mid": ["Ahri", "Viktor", "Syndra", "Orianna", "Azir", "Lissandra", 
                   "Twisted Fate", "Veigar", "Cassiopeia", "Diana", "Leblanc", 
                   "Aurelion Sol", "Ryze", "Xerath", "Zoe", "Malzahar", "Taliyah", 
                   "Anivia", "Annie", "Aurora", "Hwei", "Kennen", "Kayle",
                   "Vladimir", "Cho’Gath", "Mel", "Nunu", "Nautilus", "Neeko", 
                   "Singed", "Swain", "Teemo", "Velkoz", "Zyra","Vex",'AurelionSol','Chogath','Kassadin','Lux',
                   'Nidalee','Rumble','Sylas','TwistedFate'],
    "Mage - Support": ["Karma", "Seraphine", "Zilean", "Morgana", "Rakan", "Soraka", 
                       "Annie", "Neeko", "Velkoz", "Zyra", "Milio",'Veigar','Ivern','Lulu','Lux','TwistedFate'],
    "Fighter": ["Yone", "Yasuo", "Mordekaiser", "Renekton", "Darius", "Riven", 
                "Ambessa", "Graves", "Irelia", "Jayce", "Volibear", "Vladimir", 
                "Nasus", "Pantheon", "Sion", "Tryndamere", "Viego", "Lee Sin",'Gangplank','Garen','Gwen','Illaoi',
                'Kled','KSante','LeeSin','Sett','XinZhao'],
    "Marksman": ["Smolder", "Tristana", "Ezreal", "Varus", "Kaisa", "Kayle", 
                 "Akshan", "Azir", "Corki", "Zeri", "Vayne",'Caitlyn','Jhin','Kalista'],
    "Tank": ["Galio", "Cho’Gath", "Zac", "Malphite", "Volibear", "Nasus", 
             "Nautilus", "Singed", "Sion",'Chogath','DrMundo','Garen','Illaoi','KSante','Ornn','Poppy']
}

# 🚀 **3. Tính toán chỉ số mới**
df['GPM'] = df['goldEarned'] / df['timePlayed'] * 60
df['DPM'] = df['totalDamageDealtToChampions'] / df['timePlayed'] * 60

# 🚀 **4. Chọn các cột quan trọng**
features = ["GPM", "DPM", "visionScore", "wardsPlaced", "wardsKilled",
            "kills", "deaths", "totalMinionsKilled", "neutralMinionsKilled",
            "longestTimeSpentLiving", "totalDamageTaken"]

df = df[features + ['championName']]
df = df.dropna()

# 🚀 **5. Gán vai trò tướng**
def get_champion_roles(champion):
    for role, champions in champion_roles1.items():
        if champion in champions:
            return role
    return None  # Loại bỏ các dòng có None

df['championRoles'] = df['championName'].apply(get_champion_roles)
df.drop(columns=['championName'], inplace=True)

# 🚀 **6. Loại bỏ dòng có nhãn None**
df = df.dropna(subset=['championRoles'])

# 🚀 **7. Mã hóa nhãn**
label_encoder = LabelEncoder()
df['championRoles'] = label_encoder.fit_transform(df['championRoles'])

# 🚀 **8. Chuẩn hóa dữ liệu**
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# 🚀 **9. Chia train-test**
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['championRoles']), df['championRoles'], 
    test_size=0.2, random_state=42, stratify=df['championRoles']
)

# 🚀 **10. Huấn luyện mô hình**
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

best_model = None
best_accuracy = 0

for model_name, model in models.items():
    print(f"\n🔹 Huấn luyện mô hình: {model_name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print("Độ chính xác:", accuracy)
    print(classification_report(y_test, y_pred, target_names=label_encoder.inverse_transform(np.unique(y_test)), zero_division=1))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

# 🚀 **11. Lưu mô hình tốt nhất**
joblib.dump(best_model, "best_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

# 🚀 **12. Hàm dự đoán phong cách chơi**
def predict_playstyle(new_player_data):
    if not os.path.exists("best_model.pkl"):
        print("❌ Mô hình chưa được huấn luyện!")
        return None
    
    model = joblib.load("best_model.pkl")
    encoder = joblib.load("label_encoder.pkl")
    scaler = joblib.load("scaler.pkl")
    
    new_player_df = pd.DataFrame([new_player_data], columns=X_train.columns)
    new_player_df = scaler.transform(new_player_df)
    prediction = model.predict(new_player_df)
    predicted_role = encoder.inverse_transform(prediction)[0]
    
    return predicted_role, champion_roles1.get(predicted_role, "Không có tướng nào phù hợp")

# 🚀 **Lấy thông tin người chơi và dự đoán phong cách chơi**
gameName = input("Nhập Game Name: ")
tagLine = input("Nhập Tag Line (ví dụ: #NA1): ").replace("#", "")
countG = input("số match id mong muốn : ")
# Lấy PUUID
def get_puuid(gameName, tagLine):
    # Kiểm tra biến đầu vào
    if not gameName:
        raise ValueError("❌ gameName không được để trống!")
    if not tagLine:
        raise ValueError("❌ tagLine không được để trống!")
    # In giá trị để debug
    print(f"🔍 Đang lấy PUUID cho: {gameName}#{tagLine}")
    
    # Tạo URL request
    riot_api_key1 = 'RGAPI-783fe893-8000-4bab-8c51-d358011eca1a'
    root_url = 'https://americas.api.riotgames.com'
    endpoint = f'/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
    url = f"{root_url}{endpoint}?api_key={riot_api_key1}"
    
    print(f"📡 Request URL: {url}")  # Debugging

    # Gửi request
    response = requests.get(url)

    # Kiểm tra phản hồi
    if response.status_code == 200:
        return response.json().get("puuid")
    else:
        print(f"❌ Lỗi API: {response.status_code}, Thông báo: {response.text}")
        return None
  
# Lay Match ID 
def get_id_match(puuid=None, countG=None):
    riot_api_key1 = 'RGAPI-783fe893-8000-4bab-8c51-d358011eca1a'
    root_url = 'https://americas.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    response = requests.get(f"{root_url}{endpoint}?start=0&count={countG}&api_key={riot_api_key1}")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Lỗi khi lấy Match ID: {response.json()}")
        return []
def get_player_stats_from_match(match_id, gameName, tagLine):
    riot_api_key1 = 'RGAPI-783fe893-8000-4bab-8c51-d358011eca1a'
    root_url = 'https://americas.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/{match_id}'
    response = requests.get(root_url + endpoint + f'?api_key={riot_api_key1}')
    
    if response.status_code != 200:
        print(f"❌ Lỗi khi lấy dữ liệu trận đấu {match_id}: {response.status_code}")
        return None
    
    match_data = response.json()
    
    # Tìm thông tin của người chơi dựa trên gameName và tagLine
    for player in match_data['info']['participants']:
        if player['riotIdGameName'] == gameName and player['riotIdTagline'] == tagLine:
            # Tính GPM (Gold Per Minute) và DPM (Damage Per Minute)
            time_played = player['timePlayed'] if player['timePlayed'] > 0 else 1
            gpm = player['goldEarned'] / time_played * 60
            dpm = player['totalDamageDealtToChampions'] / time_played * 60

            # Trả về thông tin chỉ số
            return {
                'GPM': gpm,
                'DPM': dpm,
                'visionScore': player['visionScore'],
                'wardsPlaced': player['wardsPlaced'],
                'wardsKilled': player['wardsKilled'],
                'kills': player['kills'],
                'deaths': player['deaths'],
                'totalMinionsKilled': player['totalMinionsKilled'],
                'neutralMinionsKilled': player['neutralMinionsKilled'],
                'longestTimeSpentLiving': player['longestTimeSpentLiving'],
                'totalDamageTaken': player['totalDamageTaken']
            }
    
    print(f"❌ Không tìm thấy thông tin người chơi {gameName}#{tagLine} trong trận đấu {match_id}")
    return None

puuid = get_puuid(gameName, tagLine)
if puuid:
    print(f"✅ Lấy PUUID thành công: {puuid}")

    # Lấy danh sách Match ID
    match_ids = get_id_match(puuid,countG)

    if match_ids:
        print(f"🔹 Danh sách {countG} trận gần nhất:")
        
        # Biến để lưu trữ tổng giá trị các chỉ số
        total_stats = {
            'GPM': 0,
            'DPM': 0,
            'visionScore': 0,
            'wardsPlaced': 0,
            'wardsKilled': 0,
            'kills': 0,
            'deaths': 0,
            'totalMinionsKilled': 0,
            'neutralMinionsKilled': 0,
            'longestTimeSpentLiving': 0,
            'totalDamageTaken': 0
        }
        # Lấy thông tin người chơi cho từng trận đấu và tính toán tổng
        for match_id in match_ids:
            print(f"Trận {match_id}:")
            player_stats = get_player_stats_from_match(match_id, gameName, tagLine)
            if player_stats:
                # Cộng các giá trị của từng trận vào tổng
                for stat in total_stats:
                    total_stats[stat] += player_stats[stat]

        # Tính toán giá trị trung bình của các chỉ số
        avg_stats = {key: value / len(match_ids) for key, value in total_stats.items()}

        # In kết quả trung bình
        avg_df = pd.DataFrame([avg_stats])
        print(f"\n📊 Thông tin trung bình của người chơi {gameName}#{tagLine} trong 5 trận gần nhất:")
        print(avg_df)
        
        # Dự đoán phong cách chơi từ dữ liệu trung bình
        predicted_role, suggested_champions = predict_playstyle(avg_stats)
        print("\n🎯 Phong cách chơi dự đoán:", predicted_role)
        print("Phù hợp với vị tướng dựa trên phong cách chơi :", suggested_champions)  


🔹 Huấn luyện mô hình: RandomForest
Độ chính xác: 0.6956521739130435
                precision    recall  f1-score   support

      Assassin       0.67      0.65      0.66        82
       Fighter       0.78      0.53      0.63        40
    Mage - Mid       0.68      0.86      0.76       126
Mage - Support       1.00      0.50      0.67         4
      Marksman       1.00      0.38      0.56        13
          Tank       1.00      0.27      0.43        11

      accuracy                           0.70       276
     macro avg       0.85      0.53      0.62       276
  weighted avg       0.72      0.70      0.68       276


🔹 Huấn luyện mô hình: GradientBoosting
Độ chính xác: 0.6956521739130435
                precision    recall  f1-score   support

      Assassin       0.64      0.61      0.62        82
       Fighter       0.68      0.62      0.65        40
    Mage - Mid       0.71      0.84      0.77       126
Mage - Support       0.67      0.50      0.57         4
      Marksman

c:\Users\PC\Desktop\Python1\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
